In [2]:
import os, sys

repo_dir = '/home/kevin/repos/ReipurthBallyProject' #directory where the repo was cloned
sys.path.append(repo_dir)
from pyraf import iraf

from astropy.io import fits
import numpy as np 
import pandas as pd
from ccdproc import ImageFileCollection
from src.SubaruUtils import  subaru_reduction, obs_dirs
import shutil

## Use Case
1. open a no-bias file:
    - set region autocentroid
    - select a bunch of stars
    - save region file as \<detector\>_nobias.reg
2. run label_regions
    - save output file as \<detector\>_named.reg
3. open false image
    - load region file \<detector\>_named.reg
    - region -> select all
    - region -> centroid
    - save region file \<detector\>_false.reg
4. run regions2coo

In [44]:
import re

def reg2df(regpath):
    with open(regpath) as rp:
        region_text = rp.read()
    region_txt = region_text.split('\n')
    reg_hdr = region_txt[:3]
    circles = [c.split(',') for c in region_txt[3:] if c != '']
    x_pos = [c[0].split('(')[1] for c in circles]
    y_pos = [c[1] for c in circles]
    third_col = [c[2].split(')') for c in circles]
    radii = [c[0] for c in third_col]
    ret_dict = {'x_pos':x_pos, 'y_pos':y_pos, 'radius':radii}
    #deal with names if present
    if re.match(r'.*text=\{.*', third_col[0][1]):
        #assume that if the first one is named, they are all named
        names = [re.findall('\{(.*?)\}', s[1])[0] for s in third_col]
        ret_dict['name'] = names

    df = pd.DataFrame(ret_dict)
    return df

In [40]:
def coo2df(coopath):
    coo_df = pd.read_csv(coopath, names=['x_ref', 'y_ref','x_in','y_in'],
                         delim_whitespace=True, comment='#')
    return coo_df

In [41]:
coopath = '/home/kevin/repos/ReipurthBallyProject/SubaruCoordinateMaps/clarisse.coo'
df = coo2df(coopath)
df

,x_ref,y_ref,x_in,y_in
0,80.283126,3077.10470,55.866507,3080.15680
1,129.116370,3373.15620,104.699750,3385.36450
2,342.761790,4133.12350,312.241020,4157.54010
3,382.438800,3943.89470,358.022180,3965.25930
4,507.573970,3589.85370,486.209430,3602.06200
...,...,...,...,...
73,1321.834200,439.64542,1319.290800,416.75484
74,1555.826900,635.48706,1553.283500,615.13988
75,1891.713700,185.65651,1894.257100,162.76593
76,1489.856900,384.04155,1489.856900,363.69437


In [54]:
def df2reg(df, regpath, name_root='Region'):
    reg_hdr = ['# Region file format: DS9 version 4.1',
               'global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1',
               'physical']
    hasnames = 'name' in df.columns
    if hasnames:
        names = df.name
    else:
        names = [f'{name_root}_{i:02d}' for i in range(len(df))]

    with open(regpath,'w') as regf:
        for h in reg_hdr:
            regf.write(h+'\n')
    
        for name, reg in zip(names, df.itertuples()):
            nametxt = '{' + name + '}'
            reg_str = f'circle({reg.x_pos}, {reg.y_pos}, {reg.radius}) # text={nametxt}'
            regf.write(reg_str+'\n')

In [56]:
df = reg2df('/home/kevin/Documents/M8data/M8/nausicaa_nobias.reg')
df.radius = 8.0
df2reg(df, '/home/kevin/Documents/M8data/M8/coord_test/nausicaa_named.reg', name_root='Nausicaa')

In [63]:
df = coo2df('/home/kevin/repos/ReipurthBallyProject/SubaruCoordinateMaps/nausicaa.coo')
tempdf = df[['x_ref','y_ref']].rename(columns= {'x_ref':'x_pos','y_ref':'y_pos'})
tempdf['radius'] = 8.0
df2reg(tempdf, '/home/kevin/Documents/M8data/M8/coord_test/nausicaa_false.reg', name_root='false')
tempdf = df[['x_in','y_in']].rename(columns= {'x_in':'x_pos','y_in':'y_pos'})
tempdf['radius'] = 8.0
df2reg(tempdf, '/home/kevin/Documents/M8data/M8/coord_test/nausicaa_image.reg', name_root='image')
